In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB

In [2]:
data = pd.read_csv('D:/datasets/Spam Comments Detection/Youtube01-Psy.csv')
data.head()

,COMMENT_ID,AUTHOR,DATE,CONTENT,CLASS
0,LZQPQhLyRh80UYxNuaDWhIGQYNQ96IuCg-AYWqNPjpU,Julius NM,2013-11-07T06:20:48,"Huh, anyway check out this you[tube] channel: ...",1
1,LZQPQhLyRh_C2cTtd9MvFRJedxydaVW-2sNg5Diuo4A,adam riyati,2013-11-07T12:37:15,Hey guys check out my new channel and our firs...,1
2,LZQPQhLyRh9MSZYnf8djyk0gEF9BHDPYrrK-qCczIY8,Evgeny Murashkin,2013-11-08T17:34:21,just for test I have to say murdev.com,1
3,z13jhp0bxqncu512g22wvzkasxmvvzjaz04,ElNino Melendez,2013-11-09T08:28:43,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,z13fwbwp1oujthgqj04chlngpvzmtt3r3dw,GsMega,2013-11-10T16:05:38,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [3]:
data.shape

(350, 5)

In [4]:
data.describe()

,CLASS
count,350.000000
mean,0.500000
std,0.500716
min,0.000000
25%,0.000000
50%,0.500000
75%,1.000000
max,1.000000


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   COMMENT_ID  350 non-null    object
 1   AUTHOR      350 non-null    object
 2   DATE        350 non-null    object
 3   CONTENT     350 non-null    object
 4   CLASS       350 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 13.8+ KB


In [6]:
data.isnull().sum()

COMMENT_ID    0
AUTHOR        0
DATE          0
CONTENT       0
CLASS         0
dtype: int64

In [15]:
data = data[['CONTENT','CLASS']]

In [16]:
data.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",1
1,Hey guys check out my new channel and our firs...,1
2,just for test I have to say murdev.com,1
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,1
4,watch?v=vtaRGgvGtWQ Check this out .﻿,1


In [18]:
data['CLASS'] = data['CLASS'].map({0:'Not Spam', 1:'Spam'})

<ipython-input-18-4a1a39df8fc4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CLASS'] = data['CLASS'].map({0:'Not Spam', 1:'Spam'})


In [19]:
data.head()

,CONTENT,CLASS
0,"Huh, anyway check out this you[tube] channel: ...",Spam
1,Hey guys check out my new channel and our firs...,Spam
2,just for test I have to say murdev.com,Spam
3,me shaking my sexy ass on my channel enjoy ^_^ ﻿,Spam
4,watch?v=vtaRGgvGtWQ Check this out .﻿,Spam


#### Training a Classification Model

In [20]:
x = np.array(data["CONTENT"])
y = np.array(data["CLASS"])

cv = CountVectorizer()
x = cv.fit_transform(x)
xtrain, xtest, ytrain, ytest = train_test_split(x, y, 
                                                test_size=0.2, 
                                                random_state=42)

model = BernoulliNB()
model.fit(xtrain, ytrain)
model.score(xtest, ytest)

0.9857142857142858

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

def pred(model_object,predictors,compare):
    """1.model_object = model name
       2.predictors = data to be predicted
       3.compare = y_train"""
    predicted = model_object.predict(predictors)
    # Determine the false positive and true positive rates
    cm = pd.crosstab(compare,predicted)
    TP = cm.iloc[0,0]
    FN = cm.iloc[1,0]
    TN = cm.iloc[1,1]
    FP = cm.iloc[0,1]
    print("CONFUSION MATRIX ------->> ")
    print(cm)
    print()
    
    ##vérifier l'exactitude du modèle
    print('Classification paradox :------->>')
    print()
    print("Accuracy_score : ", accuracy_score(compare, predicted))
    print()
    print(classification_report(compare,predicted))

pred(model,xtest,ytest)

CONFUSION MATRIX ------->> 
col_0     Not Spam  Spam
row_0                   
Not Spam        27     0
Spam             1    42

Classification paradox :------->>

Accuracy_score :  0.9857142857142858

              precision    recall  f1-score   support

    Not Spam       0.96      1.00      0.98        27
        Spam       1.00      0.98      0.99        43

    accuracy                           0.99        70
   macro avg       0.98      0.99      0.99        70
weighted avg       0.99      0.99      0.99        70



In [27]:
test = "Try this one: https://networkingfunda.com/" 
data = cv.transform([test]).toarray()
print(model.predict(data))

['Spam']


In [28]:
test = "I'm pretty sure this will be the best $7 you spend in a long time!" 
data = cv.transform([test]).toarray()
print(model.predict(data)) 

['Not Spam']
